In [ ]:
!pip install python-telegram-bot --upgrade
!pip install gigachat
!pip install -U langchain-gigachat

In [ ]:
import asyncio
import logging
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain_gigachat.chat_models import GigaChat

# === Настройка GigaChat ===
credentials = "MDE5OTc4MTEtM2NjMS03ODNkLTkxYzAtMmM4MzZhN2UxNzM2OmE0YmY3NTdkLTkxNjItNGRjNi04ZDA1LTBiOTM4ZTRjM2JjOA=="
giga = GigaChat(
    model="GigaChat-2-Max",
    credentials=credentials,
    verify_ssl_certs=False
)

# Промпт (без return_messages!)
prompt_template = """Ты бот, который создает планы обучения для людей. Учитывай контекст предыдущих сообщений. При запросе пользователя плана обучения, учитывай требования к плану обучения от пользователя. План должен выглядеть в виде таблицы.
{history}
Human: {input}
Assistant:"""
PROMPT = PromptTemplate(input_variables=["history", "input"], template=prompt_template)

# Хранилище цепочек (по user_id)
user_chains = {}

def get_chain(user_id: int) -> ConversationChain:
    if user_id not in user_chains:
        memory = ConversationBufferMemory()  # ← без return_messages!
        chain = ConversationChain(
            llm=giga,
            memory=memory,
            prompt=PROMPT,
            verbose=False
        )
        user_chains[user_id] = chain
    return user_chains[user_id]

# === Telegram обработчики ===
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text(
        "Привет! Я помогу составить план обучения 📚\n"
        "Напишите, что хотите изучать и за какое время."
    )

async def clear(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_id = update.effective_user.id
    if user_id in user_chains:
        del user_chains[user_id]
    await update.message.reply_text("Память очищена!")

async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user = update.effective_user
    text = update.message.text

    try:
        await update.message.reply_text("Готовлю план... ⏳")
        chain = get_chain(user.id)

        # Выполняем синхронный вызов в отдельном потоке
        loop = asyncio.get_event_loop()
        response = await loop.run_in_executor(None, chain.predict, text)

        await update.message.reply_text(response.strip())
    except Exception as e:
        logging.exception("Ошибка в обработке сообщения")
        await update.message.reply_text("Извините, произошла ошибка. Попробуйте позже.")

# === Запуск ===
async def main():
    TOKEN = "8250825015:AAE9nIh5RLmbjNFl2yS0m3sBUKhfi3VJXd8"  # ← замените!
    app = Application.builder().token(TOKEN).build()

    app.add_handler(CommandHandler("start", start))
    app.add_handler(CommandHandler("clear", clear))
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

    logging.info("Бот запущен...")
    await app.initialize()
    await app.start()
    await app.updater.start_polling()

    try:
        while True:
            await asyncio.sleep(10)
    except KeyboardInterrupt:
        pass
    finally:
        await app.stop()
        await app.shutdown()

In [ ]:
await main()

ERROR:telegram.ext.Updater:Exception happened while polling for updates.
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/telegram/request/_baserequest.py", line 305, in _request_wrapper
    code, payload = await self.do_request(
                    ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/telegram/request/_httpxrequest.py", line 254, in do_request
    raise RuntimeError("This HTTPXRequest is not initialized!")
RuntimeError: This HTTPXRequest is not initialized!

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/telegram/ext/_utils/networkloop.py", line 134, in network_retry_loop
    await do_action()
  File "/usr/local/lib/python3.12/dist-packages/telegram/ext/_utils/networkloop.py", line 127, in do_action
    action_cb_task.result()
  File "/usr/local/lib/python3.12/dist-packages/telegram/ext/_updater.py", line 340, in 

RuntimeError: This Updater is still running!

Другой варик

In [1]:
# Ячейка 1: Установка (выполните один раз)
!pip install python-telegram-bot langchain langchain-gigachat --upgrade -q

In [2]:
# Ячейка 2: Основной код бота

import asyncio
import logging
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain_gigachat.chat_models import GigaChat

# === Настройка GigaChat ===
# ⚠️ Замените на ваш credentials (лучше через getpass, но для Colab можно так)
CREDENTIALS = "MDE5OTc4MTEtM2NjMS03ODNkLTkxYzAtMmM4MzZhN2UxNzM2OmE0YmY3NTdkLTkxNjItNGRjNi04ZDA1LTBiOTM4ZTRjM2JjOA=="

giga = GigaChat(
    model="GigaChat-2-Max",
    credentials=CREDENTIALS,
    verify_ssl_certs=False
)

# === Промпт БЕЗ return_messages! ===
PROMPT_TEMPLATE = """Ты бот, который создает планы обучения для людей. Учитывай контекст предыдущих сообщений. При запросе пользователя плана обучения, учитывай требования к плану обучения от пользователя. План должен выглядеть в виде таблицы.
{history}
Human: {input}
Assistant:"""

prompt = PromptTemplate(input_variables=["history", "input"], template=PROMPT_TEMPLATE)

# === Хранилище цепочек по user_id ===
user_chains = {}

def get_chain(user_id: int) -> ConversationChain:
    if user_id not in user_chains:
        memory = ConversationBufferMemory()  # ← ВАЖНО: без return_messages=True
        chain = ConversationChain(
            llm=giga,
            memory=memory,
            prompt=prompt,
            verbose=False
        )
        user_chains[user_id] = chain
    return user_chains[user_id]

# === Обработчики Telegram ===
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text(
        "Привет! 📚 Я помогу составить персональный план обучения.\n"
        "Напишите, что хотите изучать, за какое время и сколько времени в день можете уделять."
    )

async def clear(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_id = update.effective_user.id
    if user_id in user_chains:
        del user_chains[user_id]
    await update.message.reply_text("Память очищена! Начнём заново.")

async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user = update.effective_user
    text = update.message.text

    try:
        await update.message.reply_text("Составляю план... ⏳")
        chain = get_chain(user.id)

        # Выполняем синхронный вызов в пуле потоков
        loop = asyncio.get_event_loop()
        response = await loop.run_in_executor(None, chain.predict, text)

        await update.message.reply_text(response.strip())
    except Exception as e:
        logging.exception("Ошибка при генерации ответа")
        await update.message.reply_text("Извините, не удалось сгенерировать ответ. Попробуйте позже.")

# === Запуск бота ===
async def main():
    TELEGRAM_TOKEN = "8250825015:AAE9nIh5RLmbjNFl2yS0m3sBUKhfi3VJXd8"  # ← ОБЯЗАТЕЛЬНО замените!

    # Создаём НОВОЕ приложение
    app = Application.builder().token(TELEGRAM_TOKEN).build()

    app.add_handler(CommandHandler("start", start))
    app.add_handler(CommandHandler("clear", clear))
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

    logging.info("Бот запущен...")
    await app.initialize()
    await app.start()
    await app.updater.start_polling()

    try:
        while True:
            await asyncio.sleep(10)
    except KeyboardInterrupt:
        pass
    finally:
        await app.stop()
        await app.shutdown()

In [3]:
# Ячейка 3: ЗАПУСК (выполняйте ТОЛЬКО ПОСЛЕ Restart runtime!)

# Защита от повторного запуска
import sys
if 'bot_running' not in globals():
    bot_running = True
    await main()
else:
    print("⚠️ Бот уже был запущен. Перезапустите среду (Runtime → Restart runtime).")

/tmp/ipython-input-1689849996.py:36: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()  # ← ВАЖНО: без return_messages=True
/tmp/ipython-input-1689849996.py:37: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  chain = ConversationChain(
ERROR:root:Ошибка при генерации ответа
Traceback (most recent call last):
  File "/tmp/ipython-input-1689849996.py", line 69, in handle_message
    response = await loop.run_in_executor(None, chain.predict, text)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/concurrent/futures/thread.py", line 59, in run
    result = self.fn(*self.args, **self.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-p

RuntimeError: This Updater is still running!